# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117817e+02     1.792349e+00
 * time: 0.3033459186553955
     1     1.042619e+01     1.214513e+00
 * time: 1.9471888542175293
     2    -1.198949e+01     1.299100e+00
 * time: 2.50280499458313
     3    -3.408437e+01     1.111515e+00
 * time: 3.6065449714660645
     4    -4.761871e+01     8.242168e-01
 * time: 4.415462970733643
     5    -5.678143e+01     3.053850e-01
 * time: 5.235323905944824
     6    -5.963106e+01     2.016425e-01
 * time: 5.966808080673218
     7    -6.078083e+01     6.287355e-02
 * time: 6.515851974487305
     8    -6.126370e+01     7.415019e-02
 * time: 7.06993293762207
     9    -6.152894e+01     5.523532e-02
 * time: 7.6248779296875
    10    -6.174504e+01     3.721089e-02
 * time: 8.171859979629517
    11    -6.191352e+01     2.491940e-02
 * time: 8.724031925201416
    12    -6.201431e+01     2.045228e-02
 * time: 9.269917964935303
    13    -6.207864e+01     1.504394e-02
 * time: 9.816552877426147
    14  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671078
    AtomicLocal         -18.8557717
    AtomicNonlocal      14.8522665
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485387 
    Xc                  -19.3336826

    total               -62.261666454035
